<a href="https://colab.research.google.com/github/JuanPablo-Ramos/ProyectoKaggle/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**UDEA/ai4eng 20251 - Pruebas Saber Pro Colombia**

#Importamos las liberías necesarias



*   **Pandas y Numpy:** manipulación de datos.

*   **SimpleImputer:** para rellenar los valores faltantes.

*   **OneHotEncoder, OrdinalEncoder:** para convertir variables categóricas en números.

*   **StandardScaler:** para normalizar las variables numéricas (muy útil en muchos modelos).



In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

#Cargamos los datos de `train.csv`

In [ ]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("Se cargaron los datos correctamente", z.shape)

Se cargaron los datos correctamente (692500, 21)


#Realizamos algunas verificaciones dentro de `train.csv` para analizar el tipo y cantidad de datos

In [ ]:
sum(z.FAMI_EDUCACIONMADRE.isna()), sum(z.ESTU_VALORMATRICULAUNIVERSIDAD.isna())

(23664, 6287)

In [ ]:
z.FAMI_EDUCACIONMADRE.values[z.FAMI_EDUCACIONMADRE.isna()] = 'no info'
z.ESTU_VALORMATRICULAUNIVERSIDAD.values[z.ESTU_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'
sum(z.FAMI_EDUCACIONMADRE.isna()), sum(z.ESTU_VALORMATRICULAUNIVERSIDAD.isna())

(0, 0)

In [ ]:
z.isna().sum()

,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,0
ESTU_HORASSEMANATRABAJA,30857
FAMI_ESTRATOVIVIENDA,32137
FAMI_TIENEINTERNET,26629
FAMI_EDUCACIONPADRE,23178
FAMI_TIENELAVADORA,39773


In [ ]:
z = z.fillna('no info')
z.isna().sum()

,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,0
ESTU_HORASSEMANATRABAJA,0
FAMI_ESTRATOVIVIENDA,0
FAMI_TIENEINTERNET,0
FAMI_EDUCACIONPADRE,0
FAMI_TIENELAVADORA,0


In [ ]:
cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Menos de 500 mil': .250,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Más de 7 millones': 7.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'No pagó matrícula': 0,
 'no info': -1}

z.ESTU_VALORMATRICULAUNIVERSIDAD = np.r_[[cmap[i] for i in z.ESTU_VALORMATRICULAUNIVERSIDAD]]
z.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,6287


In [ ]:
# Dimensiones y columnas
print("Filas y columnas:", z.shape)
print("Columnas:", z.columns.tolist())

Filas y columnas: (692500, 21)
Columnas: ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


#Separación del objetivo y elminación de columnas irrelevantes

*   `RENDIMIENTO_GLOBAL` será nuestro target y variable a predecir, por lo tanto, la separamos.




*   `ID` Y `PERIODO` son columnas que no aportan valor predictivo:
    `ID` es un identificador único y `PERIODO` pordría introducir un sesgo temporal



In [ ]:
# Eliminar columnas de ID y PERIODO
z = z.drop(columns=["ID", "PERIODO"])
target = z["RENDIMIENTO_GLOBAL"]
z = z.drop(columns=["RENDIMIENTO_GLOBAL"])

#Identificación de tipos de columnas

En este campo lo que hacemos es disntinguir entre cuales son nuestras variables categóricas (`categorical_cols`) y cuales son numéricas (`numerical_cols`)

In [ ]:
categorical_cols = z.select_dtypes(include="object").columns.tolist()
numerical_cols = z.select_dtypes(include=["int64", "float64"]).columns.tolist()
print("Categorical:", categorical_cols)
print("Numerical:", numerical_cols)


Categorical: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']
Numerical: ['ESTU_VALORMATRICULAUNIVERSIDAD', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


#Imputación de datos faltantes



*   Rellenamos las variables categòricas con el valor más frecuente (moda).

*   Rellenamos las variables numéricas con la media, esto para no sesgar mucho los datos.



In [ ]:
#Imputación datos faltantes
cat_imputer = SimpleImputer(strategy="most_frequent")
num_imputer = SimpleImputer(strategy="mean")

z[categorical_cols] = cat_imputer.fit_transform(z[categorical_cols])
z[numerical_cols] = num_imputer.fit_transform(z[numerical_cols])




#Codificación de variables categóricas

Utilizamos `OneHotEncoder` para generar columnas con todas las categorías y tener una codificaciòn más sencilla por cada categoría.

In [ ]:
# OneHotEncoding
z_categorical = pd.DataFrame(cat_imputer.fit_transform(z[categorical_cols]), columns=categorical_cols)

onehot = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
z_categorical_encoded = pd.DataFrame(onehot.fit_transform(z_categorical), columns=onehot.get_feature_names_out())

#Normalización de columnas numéricas

Necesitamos que los datos estén en una escala similar, esto para tener una mayor facilidad computacional a la hora de compararlos:

Usamos `StandardScaler` para convertir estos valores en:


*   Media -> 0
*   Desviación estándar -> 1



In [ ]:
scaler = StandardScaler()
z_num_scaled = pd.DataFrame(scaler.fit_transform(z[numerical_cols]), columns=numerical_cols)

#Unión de las variables preprocesadas

Finalmente, concatenamos todas las columnas que se oricesaron en un solo Dataframe, y al final le agregamos la columna objetico para que el dataset estè listo para el modelado.

In [ ]:
# Concatenar todo
z_final = pd.concat([z_categorical_encoded, z_num_scaled], axis=1)
z_final["RENDIMIENTO_GLOBAL"]= target.values
z_final.head()

,ESTU_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,ESTU_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,ESTU_PRGM_ACADEMICO_ACUICULTURA,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,ESTU_PRGM_ACADEMICO_ADMINISTRACION & SERVICIO,ESTU_PRGM_ACADEMICO_ADMINISTRACION AERONAUTICA,ESTU_PRGM_ACADEMICO_ADMINISTRACION AGROPECUARIA,ESTU_PRGM_ACADEMICO_ADMINISTRACION AMBIENTAL,ESTU_PRGM_ACADEMICO_ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES,...,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_no info,ESTU_VALORMATRICULAUNIVERSIDAD,coef_1,coef_2,coef_3,coef_4,RENDIMIENTO_GLOBAL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.482661,0.437002,-0.556223,0.813978,0.060296,medio-alto
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.191253,0.346934,-0.481341,0.508180,0.016142,bajo
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.191253,0.232301,-0.492038,0.729034,0.016142,bajo
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.836957,1.771650,-0.941332,-0.171371,-1.072993,alto
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.191253,0.387874,-0.299484,0.389259,0.457683,medio-bajo
